In [1]:
# import dependencies
from bs4 import BeautifulSoup
import requests
import re
import timeit
import pandas as pd

### Load Data

In [2]:
file_path = '../Data/database.csv'
stats_df = pd.read_csv(file_path)
stats_df['game_id'] = stats_df['game_id']+1
stats_df.head(5)

,game_id,map_name,map_duration,player_name,agent,team_abrev,kills,deaths,assists,adr,fk,fd,clutches,aces,score_dif,round_total
0,1,Ascent,1:05:24,mazin,Omen,MIBR,20.0,15.0,10.0,161.0,3.0,1.0,1,0,-4,22
1,1,Ascent,1:05:24,artzin,Yoru,MIBR,18.0,15.0,9.0,139.0,7.0,0.0,0,0,-4,22
2,1,Ascent,1:05:24,ShahZaM,Sova,MIBR,12.0,14.0,9.0,122.0,2.0,1.0,0,0,-4,22
3,1,Ascent,1:05:24,liazzi,Killjoy,MIBR,13.0,14.0,5.0,126.0,0.0,2.0,0,0,-4,22
4,1,Ascent,1:05:24,Pa1nt,Iso,MIBR,11.0,17.0,6.0,98.0,2.0,4.0,0,0,-4,22


In [3]:
file_path = '../Data/series_df.csv'
series_df = pd.read_csv(file_path)
# Add series_id column starting at 1
series_df['series_id'] = range(1, len(series_df) + 1)
series_df.head(5)

,home_team,away_team,home_round_difference,away_round_difference,num_maps,series_id
0,MIBR,LEV,-10,10,2,1
1,SEN,NRG,7,-7,2,2
2,FUR,100T,4,-4,2,3
3,LOUD,EG,-3,3,3,4
4,G2,C9,-1,1,3,5


In [4]:
file_path = '../Data/scores_data.csv'
games_df = pd.read_csv(file_path)
games_df['game_id'] = games_df['game_id']+1
# Drop rows where both home_score and away_score are 0
games_df = games_df[~((games_df['home_score'] == 0) & (games_df['away_score'] == 0))]
games_df.tail()

,game_id,map_name,home_team,away_team,map_duration,home_score,away_score
43,44,Icebox,NRG,C9,49:20,10,13
44,45,Sunset,NRG,C9,59:18,12,14
45,46,Lotus,EG,100T,1:06:50,10,13
46,47,Bind,EG,100T,55:39,13,10
47,48,Ascent,EG,100T,46:01,7,13


In [5]:
file_path = '../Data/player_data.csv'
player_df = pd.read_csv(file_path)
# Add series_id column starting at 1
player_df['player_id'] = range(1, len(player_df) + 1)
player_df.head()

,player_name,team_abrev,player_id
0,Apoth,EG,1
1,Asuna,100T,2
2,Boostio,100T,3
3,C0M,LEV,4
4,Cryocells,100T,5


## Create player_stats

##### player_id, series_id, game_id, agent, kills, deaths, assists, fk, fd, clutches, aces, adr, points

In [6]:
# Merge the dataframes on player_name and team_abrev
stats_df_with_id = stats_df.merge(player_df, on=['player_name', 'team_abrev'], how='left')
stats_df_with_id

,game_id,map_name,map_duration,player_name,agent,team_abrev,kills,deaths,assists,adr,fk,fd,clutches,aces,score_dif,round_total,player_id
0,1,Ascent,1:05:24,mazin,Omen,MIBR,20.0,15.0,10.0,161.0,3.0,1.0,1,0,-4,22,41.0
1,1,Ascent,1:05:24,artzin,Yoru,MIBR,18.0,15.0,9.0,139.0,7.0,0.0,0,0,-4,22,26.0
2,1,Ascent,1:05:24,ShahZaM,Sova,MIBR,12.0,14.0,9.0,122.0,2.0,1.0,0,0,-4,22,20.0
3,1,Ascent,1:05:24,liazzi,Killjoy,MIBR,13.0,14.0,5.0,126.0,0.0,2.0,0,0,-4,22,40.0
4,1,Ascent,1:05:24,Pa1nt,Iso,MIBR,11.0,17.0,6.0,98.0,2.0,4.0,0,0,-4,22,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,48,Ascent,46:01,Cryocells,Jett,100T,25.0,13.0,1.0,199.0,7.0,2.0,0,0,6,20,5.0
476,48,Ascent,46:01,eeiu,Sova,100T,19.0,9.0,5.0,161.0,2.0,1.0,0,0,6,20,31.0
477,48,Ascent,46:01,Asuna,Kayo,100T,14.0,13.0,24.0,132.0,2.0,1.0,1,0,6,20,2.0
478,48,Ascent,46:01,bang,Omen,100T,14.0,12.0,12.0,137.0,1.0,1.0,0,0,6,20,28.0


In [7]:
# Merge the dataframes on game_id
stats_df_complete = stats_df_with_id.merge(games_df, on=['game_id', 'map_name'], how='left')
stats_df_complete

,game_id,map_name,map_duration_x,player_name,agent,team_abrev,kills,deaths,assists,adr,...,clutches,aces,score_dif,round_total,player_id,home_team,away_team,map_duration_y,home_score,away_score
0,1,Ascent,1:05:24,mazin,Omen,MIBR,20.0,15.0,10.0,161.0,...,1,0,-4,22,41.0,MIBR,LEV,1:05:24,9,13
1,1,Ascent,1:05:24,artzin,Yoru,MIBR,18.0,15.0,9.0,139.0,...,0,0,-4,22,26.0,MIBR,LEV,1:05:24,9,13
2,1,Ascent,1:05:24,ShahZaM,Sova,MIBR,12.0,14.0,9.0,122.0,...,0,0,-4,22,20.0,MIBR,LEV,1:05:24,9,13
3,1,Ascent,1:05:24,liazzi,Killjoy,MIBR,13.0,14.0,5.0,126.0,...,0,0,-4,22,40.0,MIBR,LEV,1:05:24,9,13
4,1,Ascent,1:05:24,Pa1nt,Iso,MIBR,11.0,17.0,6.0,98.0,...,0,0,-4,22,17.0,MIBR,LEV,1:05:24,9,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,48,Ascent,46:01,Cryocells,Jett,100T,25.0,13.0,1.0,199.0,...,0,0,6,20,5.0,EG,100T,46:01,7,13
476,48,Ascent,46:01,eeiu,Sova,100T,19.0,9.0,5.0,161.0,...,0,0,6,20,31.0,EG,100T,46:01,7,13
477,48,Ascent,46:01,Asuna,Kayo,100T,14.0,13.0,24.0,132.0,...,1,0,6,20,2.0,EG,100T,46:01,7,13
478,48,Ascent,46:01,bang,Omen,100T,14.0,12.0,12.0,137.0,...,0,0,6,20,28.0,EG,100T,46:01,7,13


In [8]:
# merge series_df on home_team and away_team
final_df = pd.merge(stats_df_complete, series_df, on=['home_team', 'away_team'])

In [9]:
# Dropping columns from final_df
columns_to_drop = ['map_duration_x', 'round_total', 'home_round_difference', 'away_round_difference', 'num_maps', 'map_duration_y']
final_df = final_df.drop(columns=columns_to_drop)

In [10]:
# reorder the columns
new_order = ['player_id', 'series_id', 'game_id', 'agent', 'kills', 'deaths', 'assists', 'fk', 'fd', 
             'clutches', 'aces', 'adr']
final_df = final_df[new_order]

In [11]:
agent_roles = {
    'Brimstone': 'Controller',
    'Viper': 'Controller',
    'Omen': 'Controller',
    'Killjoy': 'Sentinel',
    'Cypher': 'Sentinel',
    'Sova': 'Initiator',
    'Sage': 'Sentinel',
    'Phoenix': 'Duelist',
    'Jett': 'Duelist',
    'Reyna': 'Duelist',
    'Raze': 'Duelist',
    'Breach': 'Initiator',
    'Skye': 'Initiator',
    'Yoru': 'Duelist',
    'Astra': 'Controller',
    'KAY/O': 'Initiator',
    'Chamber': 'Sentinel',
    'Neon': 'Duelist',
    'Fade': 'Initiator',
    'Harbor': 'Controller',
    'Gekko': 'Initiator',
    'Deadlock': 'Sentinel',
    'Iso': 'Duelist',
    'Clove': 'Controller'
}

# create a function to loop through the agents
def get_agent_role(agent_name):
    return agent_roles.get(agent_name, 'Unknown')

# apply the function
# Apply the function to create the agent_role column
final_df['agent_role'] = final_df['agent'].apply(get_agent_role)

In [12]:
# Define points allocation dictionary
points_allocation = {
    "kills": 1,
    "deaths": -0.5,
    "assists": 0.5,
    "fk": 2,
    "fd": -1,
    "clutches": 2,
    "aces": 3,
    "adr": 0.1
}

# Function to calculate points
def calculate_points(row):
    points = (
        row['kills'] * points_allocation['kills'] +
        row['deaths'] * points_allocation['deaths'] +
        row['assists'] * points_allocation['assists'] +
        row['fk'] * points_allocation['fk'] +
        row['fd'] * points_allocation['fd'] +
        row['clutches'] * points_allocation['clutches'] +
        row['aces'] * points_allocation['aces'] +
        row['adr'] * points_allocation['adr']
    )
    return points

# Apply the function to calculate points
final_df['points'] = final_df.apply(calculate_points, axis=1)

In [13]:
new_order = ['player_id', 'series_id', 'game_id', 'agent', 'agent_role', 'kills', 'deaths', 'assists', 'fk', 'fd', 'clutches', 'aces', 'adr', 'points']

final_df = final_df[new_order]

In [14]:
final_df['player_id'] = final_df['player_id'].astype(float).astype(int)

In [15]:
final_df['game_id'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 39, 40, 41, 42,
       43, 44, 45, 46, 47, 48], dtype=int64)

In [16]:
# Save DataFrame to CSV file
final_df.to_csv('../Data/import_tables/player_stats.csv', index=False)

## Create player table

##### player_name, team_abrev

In [17]:
player_df.head()

,player_name,team_abrev,player_id
0,Apoth,EG,1
1,Asuna,100T,2
2,Boostio,100T,3
3,C0M,LEV,4
4,Cryocells,100T,5


In [18]:
player_df = player_df.drop(columns=['player_id'])
player_df.head(3)

,player_name,team_abrev
0,Apoth,EG
1,Asuna,100T
2,Boostio,100T


In [19]:
# Save DataFrame to CSV file
player_df.to_csv('../Data/import_tables/player.csv', index=False)

## Create games table

##### series_id, map_name, home_team, away_team, map_duration, home_score, away_score

In [20]:
games_df.tail(3)

,game_id,map_name,home_team,away_team,map_duration,home_score,away_score
45,46,Lotus,EG,100T,1:06:50,10,13
46,47,Bind,EG,100T,55:39,13,10
47,48,Ascent,EG,100T,46:01,7,13


In [21]:
# Replace 'KRÜ' with 'KRU' in both 'home_team' and 'away_team' columns
games_df['home_team'] = games_df['home_team'].replace('KRÜ', 'KRU')
games_df['away_team'] = games_df['away_team'].replace('KRÜ', 'KRU')

In [22]:
# Merge the DataFrames
games_df_with_series = games_df.merge(series_df, on=['home_team', 'away_team'], how='left')

In [23]:
games_df_with_series = games_df_with_series.drop(columns=['home_round_difference', 'away_round_difference', 'num_maps'])
games_df_with_series.tail()

,game_id,map_name,home_team,away_team,map_duration,home_score,away_score,series_id
43,44,Icebox,NRG,C9,49:20,10,13,19
44,45,Sunset,NRG,C9,59:18,12,14,19
45,46,Lotus,EG,100T,1:06:50,10,13,20
46,47,Bind,EG,100T,55:39,13,10,20
47,48,Ascent,EG,100T,46:01,7,13,20


In [24]:
new_order = ['game_id','series_id', 'map_name', 'home_team', 'away_team', 'map_duration', 'home_score', 'away_score']
games_df_with_series = games_df_with_series[new_order]
games_df_with_series

,game_id,series_id,map_name,home_team,away_team,map_duration,home_score,away_score
0,1,1,Ascent,MIBR,LEV,1:05:24,9,13
1,2,1,Icebox,MIBR,LEV,45:04,7,13
2,3,2,Lotus,SEN,NRG,59:21,13,8
3,4,2,Sunset,SEN,NRG,1:00:54,14,12
4,5,3,Icebox,FUR,100T,1:12:51,14,12
5,6,3,Haven,FUR,100T,1:00:50,16,14
6,7,4,Lotus,LOUD,EG,52:39,13,7
7,8,4,Sunset,LOUD,EG,40:31,6,13
8,9,4,Icebox,LOUD,EG,55:13,11,13
9,10,5,Ascent,G2,C9,1:05:05,13,11


In [25]:
games_df_no_id = games_df_with_series.drop(columns=['game_id'])

In [26]:
games_df_no_id

,series_id,map_name,home_team,away_team,map_duration,home_score,away_score
0,1,Ascent,MIBR,LEV,1:05:24,9,13
1,1,Icebox,MIBR,LEV,45:04,7,13
2,2,Lotus,SEN,NRG,59:21,13,8
3,2,Sunset,SEN,NRG,1:00:54,14,12
4,3,Icebox,FUR,100T,1:12:51,14,12
5,3,Haven,FUR,100T,1:00:50,16,14
6,4,Lotus,LOUD,EG,52:39,13,7
7,4,Sunset,LOUD,EG,40:31,6,13
8,4,Icebox,LOUD,EG,55:13,11,13
9,5,Ascent,G2,C9,1:05:05,13,11


In [27]:
games_df_no_id.to_csv('../Data/import_tables/games.csv', index=False)

## Create series table

##### split, week, home_team, away_team, home_round_difference, away_round_difference, num_maps

In [28]:
# Adding a split column
series_df['split'] = 2

# Adding a static week column
series_df['week'] = 1

In [29]:
series_df.tail(3)

,home_team,away_team,home_round_difference,away_round_difference,num_maps,series_id,split,week
17,MIBR,FUR,-9,9,2,18,2,1
18,NRG,C9,5,-5,3,19,2,1
19,EG,100T,-6,6,3,20,2,1


In [30]:
# Dictionary mapping series_id to week number
series_week_dict = {
    1: 1, 2: 1, 3: 1, 4: 1, 5: 1,
    6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 
    16: 3, 17: 3, 18: 3, 19: 3, 20: 3
}

In [31]:
# Update the week column based on series_week_dict
series_df['week'] = series_df['series_id'].map(series_week_dict)

In [32]:
new_order = ['series_id', 'split', 'week', 'home_team', 'away_team', 'home_round_difference', 'away_round_difference', 'num_maps']
series_df = series_df[new_order]

In [33]:
series_df = series_df.drop(columns=['series_id'])
series_df.head(3)

,split,week,home_team,away_team,home_round_difference,away_round_difference,num_maps
0,2,1,MIBR,LEV,-10,10,2
1,2,1,SEN,NRG,7,-7,2
2,2,1,FUR,100T,4,-4,2


In [34]:
# Save DataFrame to CSV file
series_df.to_csv('../Data/import_tables/series.csv', index=False)